<a href="https://colab.research.google.com/github/joshmcgee1/SangerSeqAlign/blob/main/Sanger_Sequence_Align.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Instructions: Select Runtime -> Run All, Upload GenBank reference file for plasmid that you wish to have reads aligned to, upload .abi files for Sanger Sequencing Reads

#GenBank Reference File Upload (Vector Template for Alignment)

In [ ]:
from google.colab import files

uploaded = files.upload()
fn = uploaded.keys()
for fn in uploaded.keys():
  print(fn)

#Upload Sanger Read Files (abi format) OK to upload

In [ ]:
from google.colab import files

seq_files = files.upload()

#Run Code to Receive Alignment Results

In [ ]:
!pip install easygui
!pip install Bio
!pip install pandas

import pandas as pd
import easygui
from Bio import SeqIO
from Bio import pairwise2
from Bio.pairwise2 import format_alignment

record_ref = SeqIO.read(fn, "genbank")

feature_dict = {"Name": [], "Type":[], "Start": [], "End": [], "Strand": []}
feature_start = []
feature_end = []

for feature in record_ref.features:
    if feature.type == "CDS" or feature.type == "misc_feature" or feature.type == "primer":
        feature_dict["Name"].append(feature.qualifiers['label'])
        feature_dict["Type"].append(feature.type)
        feature_dict["Start"].append(feature.location.start.position)
        feature_dict["End"].append(feature.location.end.position)
        feature_dict["Strand"].append(feature.strand)
        feature_start.append(feature.location.start.position)
        feature_end.append(feature.location.end.position)

ref_seq = record_ref.seq

print('Initializing.....****************************************************************************************')
print('reference file: ', record_ref.id ,' length: ', len(ref_seq))
print('sequence: ', ref_seq)
print()
print('----- Extracted Features from Reference ------')
df = pd.DataFrame(feature_dict)
print(df)

print()
print('*********************************************************************************************************')
sequences = []
sequence_list = [record_ref, ]
results_list = {"File Name": [], "Read Length": [], "% Match": [], "Strand": [], "Matched Features": [], "Sequence": []}

skip_reverse = 0
strand = 1
idx = 1
for file in seq_files:
    record = SeqIO.read(file, "abi-trim")
    sequence_list.append(record)
    seq = record.seq
    if seq == 'NNNNN':
        print('Sequencing Failed for ID:', record.id)
        results_list["File Name"].append(record.id)
        results_list["Read Length"].append('Failed')
        results_list["% Match"].append('Failed')
        results_list["Strand"].append('Failed')
        results_list["Matched Features"].append('Failed')
        results_list["Sequence"].append('Failed')
        print('*********************************************************************************************************')
        print()
    else:
        reverse = record.seq.reverse_complement()
        print()
        print('******************** New Sequencing Record ',idx, '/', len(seq_files),' **************************************************************')
        print('file: ', record.id, ' read length: ', len(seq))
        print('extracted seq: ', seq)
        sequences = [record_ref, record]

        print('Forward Alignment for ID: ', record.id)
        test_alignments = pairwise2.align.localms(seq, ref_seq, 1, -2, -5, -4, one_alignment_only=True)
        seq_length = min(len(seq), len(ref_seq))
        matches = test_alignments[0][2]
        percent_match = (matches / seq_length) * 100

        print(format_alignment(*test_alignments[0], full_sequences=False))

        results_list["File Name"].append(record.id)
        results_list["Read Length"].append(len(seq))

        fwd_start = test_alignments[0][3]
        fwd_end = test_alignments[0][4]

        matched_list = {"Name": []}
        if percent_match > 90:
            results_list["Sequence"].append(seq)
            sequence_list.append(record)
            percent_match = "{:.2f}".format(percent_match)
            results_list["% Match"].append(percent_match)
            results_list["Strand"].append('FWD')
            for i in range(0, len(feature_start)):
              if fwd_start in range(feature_dict["Start"][i], feature_dict["End"][i]) or fwd_end in range(
                  feature_dict["Start"][i], feature_dict["End"][i]) and strand == feature_dict["Strand"][i]:
                  print('Alignment Matched to Feature: ', feature_dict["Name"][i],' Type: ', feature_dict["Type"][i])
                  matched_list["Name"].append(feature_dict["Name"][i])
            results_list["Matched Features"].append(matched_list["Name"])
            matched_list = {"Name": []}
            print()
            print('% match: ', percent_match, ' matched bps: ', matches)
            skip_reverse = 1
        else:
            skip_reverse = 0
            percent_match = "{:.2f}".format(percent_match)
            print('*not a significant match*', ' % match: ', percent_match, ' matched bps: ', matches)
            fwd_failed = percent_match

        if skip_reverse != 1:
            strand = -1
            print()
            print('------- Forward Failed... Trying Reverse Alignment for ID: ', record.id, '-----------')
            test_alignments = pairwise2.align.localms(reverse, ref_seq, 1, -2, -5, -4, one_alignment_only=True)
            seq_length = min(len(reverse), len(ref_seq))
            matches = test_alignments[0][2]
            percent_match = (matches / seq_length) * 100
            print(format_alignment(*test_alignments[0], full_sequences=False))
            rev_start = test_alignments[0][3]
            rev_end = test_alignments[0][4]
            if percent_match > 90:
                results_list["Sequence"].append(seq.reverse_complement())
                sequence_list.append(record.reverse_complement())
                percent_match = "{:.2f}".format(percent_match)
                for i in range(0, len(feature_start)):
                  if rev_start in range(feature_dict["Start"][i], feature_dict["End"][i]) or rev_end in range(
                    feature_dict["Start"][i], feature_dict["End"][i]) and strand == feature_dict["Strand"][i]:
                    print('Alignment Matched to Feature: ', feature_dict["Name"][i],' Type: ', feature_dict["Type"][i])
                    matched_list["Name"].append(feature_dict["Name"][i])
                results_list["Matched Features"].append(str(matched_list["Name"]))
                matched_list = {"Name": []}

                results_list["% Match"].append(percent_match)
                results_list["Strand"].append('REV')
                print()
                print('Reverse % match: ', percent_match, ' matched bps: ', matches)
                print()
            else:
                percent_match = "{:.2f}".format(percent_match)
                print()
                if percent_match > fwd_failed:
                  results_list["% Match"].append(percent_match)
                  results_list["Strand"].append('REV')
                  results_list["Sequence"].append(seq.reverse_complement())
                  strand = -1
                  for i in range(0, len(feature_start)):
                      if rev_start in range(feature_dict["Start"][i], feature_dict["End"][i]) or rev_end in range(
                      feature_dict["Start"][i], feature_dict["End"][i]) and strand == feature_dict["Strand"][i]:
                        print('Alignment Matched to Feature: ', feature_dict["Name"][i],' Type: ', feature_dict["Type"][i])
                        matched_list["Name"].append(feature_dict["Name"][i])
                  results_list["Matched Features"].append(str(matched_list["Name"]))
                else:
                  strand = 1
                  results_list["Sequence"].append(seq)
                  for i in range(0, len(feature_start)):
                    if fwd_start in range(feature_dict["Start"][i], feature_dict["End"][i]) or fwd_end in range(
                      feature_dict["Start"][i], feature_dict["End"][i]) and strand == feature_dict["Strand"][i]:
                        matched_list["Name"].append(feature_dict["Name"][i])
                        print('Alignment Matched to Feature: ', feature_dict["Name"][i],' Type: ', feature_dict["Type"][i])
                  results_list["Matched Features"].append(str(matched_list["Name"]))
                  results_list["% Match"].append(fwd_failed)
                  results_list["Strand"].append('FWD')

                print()
                print('*not a significant match*', ' % match: ', percent_match, ' matched bps: ', matches)
                print()

            skip_reverse = 0
            strand = 1
            print()
            print('******************** End of Record ********************************************************************')
            print()
            idx = idx+1

SeqIO.write(sequence_list, "alignment.fasta", "fasta")
print()
print('------ Significant Matches Exported to alignment.fasta -----')

print()
print('----- Results Table ------')
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
df = pd.DataFrame(results_list)
print(df)

df.to_excel("results.xlsx") 
files.download("results.xlsx")
files.download("alignment.fasta")